## Побудова бейзлайнового чанкера на основі правил.

У нас уже є непоганий POS-таггер, тепер використаємо його для того, щоб витягати з речень іменникові та дієслівні фрази.

In [1]:
import string
import nltk

from tokenize_uk import tokenize_words, tokenize_sents
from perceptron_tagger.tagger import PerceptronTagger
tagger = PerceptronTagger()

`nltk.RegexpParser` дозволяє задати набір regexp-правил, які використовуються для знаходження чанків у тексті з роставленими тегами частин мови.

In [2]:
# helper functions
def pos_chunk(sent, tagger, parser):
    """
    Use tagger and chunk parser to create chunks.
    """
    vps, nps  = [], []
    pos_sent = tagger.tag(sent)
    parsed_sent = parser.parse(pos_sent)
    for s in parsed_sent.subtrees():
        if s.label() == 'VP':
            vps.append(s.leaves())
        elif s.label() == 'NP':
            nps.append(s.leaves())
    res = {'sent': sent,
           'VPs': vps,
           'NPs': nps}
    return res

def print_phrases(res_dict):
    """
    Prints res_dict from pos_chunk function.
    """
    print('Sentence:')
    sent = res_dict['sent']
    vps = res_dict['VPs']
    nps = res_dict['NPs']
    print(sent)
    if nps:
        print('Noun phrases:')
        print([' '.join([w[0] for w in np]) for np in nps])
    if vps:
        print('Verb phrases')
        print([' '.join([w[0] for w in vp]) for vp in vps])
    print('---')
    
def write_phrases(res_dict):
    """
    Make a line for tsv file.
    """
    sent = res_dict['sent']
    vps = [' '.join([w[0] for w in vp]) for vp in res_dict['VPs']]
    nps = [' '.join([w[0] for w in np]) for np in res_dict['NPs']]
    lennps = len(nps)
    lenvps = len(vps)
    vps = ', '.join(vps)
    nps = ', '.join(nps)
    line = sent+'\t'+nps+'\t'+vps+'\t'+str(lennps)+'\t'+str(lenvps)+'\n'
    return line

def phrases_to_file(list_of_dicts, file_path):
    """
    Make a tsv file.
    """
    lines = [write_phrases(res) for res in list_of_dicts]
    first = 'sentence\tNoun phrases\tVerb phrases\tn(NP)\tn(VP)\n'
    lines.insert(0, first)
    with open(file_path, 'w') as f:
        f.writelines(lines)

In [3]:
# create grammar and initialize chunk parser
grammar = r"""
NP: {<DET><ADJ><NOUN>+}
    {<DET>*<ADJ><NOUN>+}
    {<DET><ADJ>*<NOUN>+}
    {<DET><ADJ><NOUN>*<PROPN>*}
    {<DET>*<ADJ>*<NOUN>+<ADJ>*<NOUN|PROPN>+}
    {<NOUN><NOUN|PROPN>(<CCONJ><NOUN|PROPN>)?}
    {<DET>*(<ADJ>|<ADJ><CCONJ><ADJ>)*(<NOUN|PROPN><CCONJ><NOUN|PROPN>|<NOUN|PROPN>)+}
VP: {<ADV>*<VERB>+}
    {<AUX>+<VERB>+}
"""
cp = nltk.RegexpParser(grammar)

Для бейзлайнової питально-відповідальної системи я створював набір тестових питань про географічні об'єкти. Можна перевірити роботу чанкера на них.

In [4]:
with open('QA/test_questions.txt') as f:
    raw = f.read().strip()
    questions = [q.strip() for q in raw.split('\n')]

In [5]:
pos_chunk(questions[15], tagger, cp)

{'sent': 'яка густота населення Австралії',
 'VPs': [],
 'NPs': [[('яка', 'DET'), ('густота', 'NOUN'), ('населення', 'NOUN')],
  [('Австралії', 'PROPN')]]}

У цьому випадку він знайшов дві іменникові фрази і жодної дієслівної (тому що в реченні їх і не було). "Яка густота населення" та "Австралії" - окремі фрази, тому що в перспективі для знаходження відповіді на це питання краще їх розділяти, і це враховано у правилах.

Для зручнішого перегляду можна витягнути всі іменникові та дієслівні фрази у окремий файл:

In [6]:
q_chunks = []
for q in questions:
    q_chunks.append(pos_chunk(q, tagger, cp))

phrases_to_file(q_chunks, 'qchunks.tsv')

Я "протестував" чанкер на цьому файлі. Очікувано, він витягає практично все, що можливо, з цих досить простих питань. Тільки у 5 питаннях із 128 чанкер не витягнув слова, без яких система не дала б відповідь на питання, тобто recall дуже високий. Водночас precision не дуже хороший - інколи одна смислова фраза розбита на дві або відношення і об'єкт опиняються в одній фразі.

Можна створити аналогічний чанкер і на основі залежностей. Для прикладу можна взяти вже проанотовані речення.

In [7]:
import conllu
import gzip

fname = 'uk_iu-ud-dev.conllu.gz'
with gzip.open(fname, 'rb') as f:
    raw_dev = f.read().decode()

In [8]:
dev = conllu.parse(raw_dev)

In [9]:
def strip_colon(deprel):
    if not ':' in deprel:
        return deprel
    else:
        return deprel.split(':')[0]
    
grammar = r"""
NP: {<amod><nsubj>(<cc><conj>)?}
    {<nsubj><nmod><appos>?}
    {<amod>?<nmod>+<cc><amod>?<conj>}
    {<appos><nmod>(<cc><conj>)?}
    {<det>?<amod>?<nmod><amod>?<nmod><flat>?}    
    {<amod>?<nmod>+<cc><amod>?<conj>}
    {<amod>*<nmod>+<flat>?(<cc><conj>)?}
    {<amod>*<nsubj>+(<cc><conj>)?}
    {<det>?<amod>+<obj>(<cc><conj>)?}
    {<amod>?<obj>(<cc><conj>)?}
    {<flat><cc><conj>}
    {<det>?<amod>?<obl>(<cc><conj>)?}
"""
cp = nltk.RegexpParser(grammar)
sent = [(w['form'], strip_colon(w['deprel'])) for w in dev[35]]
print(cp.parse(sent))

(S
  Нині/advmod
  зростає/root
  (NP інтерес/nsubj)
  до/case
  (NP біоміметики/nmod)
  як/mark
  до/case
  (NP галузі/appos копіювання/nmod)
  (NP
    унікальних/amod
    функцій/nmod
    і/cc
    виробничих/amod
    процесів/conj)
  (NP живих/amod організмів/nmod)
  ,/punct
  застосування/conj
  цих/det
  (NP технологій/nmod)
  під/case
  (NP час/nmod розробки/nmod й/cc створення/conj)
  (NP продукції/nmod)
  ./punct)


Звісно, правила для чанкінгу обох типів підлягають подальшому покращенню, хоча зрозуміло, що правила функціонуватимуть гірше, ніж натренована на даних програма - але цих даних якраз немає (і їх, можливо, потрібно створити).